In [1]:
import pandas as pd
import pyreadstat
import doubleml as dml
import numpy as np
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression, LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from collections import defaultdict

In [2]:
df, meta = pyreadstat.read_dta("data_JC.dta")

# ACADEMIC

In [3]:
confounders = ["jcmsa_1", "jcmsa_2", "age", "race_eth_1", "race_eth_2", 
                       "race_eth_3","ntv_lang_1", "ntv_lang_2", "hh14_1", "hh14_2", 
                       "hh14_3", "hh14_4", "welf_kid_1", "welf_kid_2", "welf_kid_3", 
                       "hgc", "hgc_moth", "hgc_fath", "hgc_moth", "hgc_fath", "m_work14_0", 
                       "occ_moth_1","occ_moth_2", "occ_moth_3", "occ_moth_4", "occ_moth_5", 
                       "occ_moth_6", "occ_fath_1", "occ_fath_2", "occ_fath_3", "occ_fath_5", 
                       "occ_fath_6", "occ_fath_7", "marriage_1", "marriage_2", "marriage_3",
                       "haschld_0", "proplive_1", "proplive_2", "pregn_ra_1", "pregn_ra_2", "old", 
                       "yng", "othwith_1", "othwith_2", "othwith_3", "othwith_4", 
                       "ageparnt", "numb_hh", "r_head_0", "hhmemb_1", "hhmemb_2", "hhmemb_3", 
                       "hhmemb_4", "hhmemb_5", "hous_arr_1", "hous_arr_2", "hous_arr_3", 
                       "pay_rent_0", "hgc", "hgc_moth", "hgc_fath", "hs_d_0", "ged_d_0", "voc_d_0", 
                       "oth_deg_0", "inschool_0", "any_ed1_0", "n_ed_cat_1", "n_ed_cat_2", "monined", 
                       "reasleft_1", "reasleft_2", "reasleft_3", "reasleft_4", "reasleft_5", 
                       "reasleft_6", "reasleft_7", "reasleft_8", "rec_ed_1",  "rec_ed_2", "rec_ed_3", 
                       "rec_ed_4", "rec_ed_5", "rec_ed_6", "rec_ed_7", "typeed_r_1", "typeed_r_2", 
                       "typeed_r_3", "typeed_r_4", "typeed_r_5", "typeed_r_6", "nhrsed_r", "reased_r_1", 
                       "numbjobs", "evworkb_0", "yr_work1_0", 
                       "earn_yr", "mosinjob", "rec_job_1","rec_job_2", "rec_job_3", "rec_job_4", "occ_r_1", 
                       "occ_r_2", "occ_r_3", "occ_r_4", "occ_r_5", "occ_r_6", "occ_r_7", "occ_r_8", 
                       "hrswk_jr", "hrwager", "coop_r_0", "govprg_r_0", "leftjobr_0", "rslftjr_1", 
                       "rslftjr_2", "rslftjr_3", "rslftjr_4", "rslftjr_5", "rslftjr_6", "rslftjr_7", 
                       "rslftjr_8", "mos_afdc", "mos_othw", "mos_fs", "got_anyw_0", "mos_anyw", 
                       "gotafdc1_0", "gotothw1_0", "gotfs1_0", "hh_inc_1", "hh_inc_2", "hh_inc_3", 
                       "hh_inc_4", "pers_inc_1", "pers_inc_2", "pers_inc_3", "health_1", "health_2", 
                       "health_3", "sick_0", "typehlth_1", "typehlth_2", "typehlth_3", "typehlth_4", 
                       "typehlth_5", "py_cig_0", "ev_alchl_0", "ev_pot_0", "py_pot_0", "ev_coke_0", 
                       "py_coke_0", "ev_crack_0", "py_crack_0", "ev_hroin_0", "py_hroin_0", "ev_speed_0", 
                       "py_speed_0", "py_lsd_0", "ev_lsd_0", "ev_othdr_0", "py_othdr_0", "ev_injct_0", 
                       "drug_trt_0",  "mout_trt", "mos_trtr_1", "mos_trtr_2", "mos_trtr_3", 
                       "narrcat_1", "narrcat_2", "narrcat_3", 
                       "evarrst1_0", "rc_arrst_1", "rc_arrst_2", "rc_arrst_3", "rc_arrst_4",
                       "marrcat1_1", 
                       "marrcat1_2", "agearcat_1", "agearcat_2", "agearcat_3", "burglary_0", "robbery_0", 
                       "assault_0", "larceny_0", "drugviol_0", "othpers_0", "othmisc_0", "sercr_s1_0", 
                       "sercr_s3_0", "sercr_s4_0", "sercr_s5_0", "sercr_s6_0", "sercr_s7_0", "n_guilty", 
                       "guilty2_0", "wksjail", "pending2_0", "copplea2_0", "sercr_c1_0", "sercr_c2_0", 
                       "sercr_c3_0", "sercr_c5_0", "sercr_c6_0", "sercr_c7_0", "asslt_c2_0", "rob_c2_0", 
                       "burgl_c2_0", "larceny_0", "drviolc2_0", "othperc2_0", "othmscc2_0", "evjail2_0", 
                       "parole2_0", "hear_jc_1", "hear_jc_2", "hear_jc_3", "hear_jc_4", "hear_jc_5", 
                       "hear_jc_6", "hear_jc_7", "from_oa_0", "knew_jc_0", "info_jc_1", "info_jc_2", "info_jc_3", 
                       "info_jc_4", "info_jc_5", "info_jc_6", "info_jc_7", "r_home_0", "r_comm_0", "r_train_0", 
                       "r_crgoal_0", "r_getged_0", "r_nowork_0", "r_other_0", "mostimpr_1", "mostimpr_2", 
                       "mostimpr_3", "mostimpr_4", "mostimpr_5", "mostimpr_6", "mostimpr_7", "othimpr_1", 
                       "othimpr_2", "othimpr_3", "othimpr_4", "othimpr_5", "othimpr_6", "e_math_0", "e_read_0", 
                       "e_along_0", "e_contrl_0", "e_esteem_0", "e_spcjob_0", "e_friend_0", "knewcntr_0", 
                       "imprcntr_1", "imprcntr_2", "imprcntr_3", "imprcntr_4", "imprcntr_5", "imprcntr_6", 
                       "imprcntr_7", "imprcntr_8", "knewjob_0", "typejobb_1", "typejobb_2", "typejobb_3", 
                       "typejobb_4", "typejobb_5", "typejobb_6", "typejobb_7", "typejobb_8", "typejobb_9", 
                       "typejobb_10", "earn_cmp", "hadworry_0", "typeworr_1", "typeworr_2", "typeworr_3", 
                       "typeworr_4", "typeworr_5", "typeworr_6", "typeworr_7", "typeworr_8", "typeworr_9", 
                       "talk_par_0", "imp_par_0", "encr_par_0", "talk_rel_0", "imp_rel_0", "encr_rel_0", 
                       "talk_frd_0", "imp_frd_0", "encr_frd_0", "talk_tch_0", "imp_tch_0", "encr_tch_0", 
                       "talk_cw_0", "imp_cw_0", "encr_cw_0", "talk_pro_0", "imp_pro_0", "encr_pro_0", 
                       "talk_chl_0", "imp_chl_0", "encr_chl_0", "talk_adl_0", "imp_adl_0", "encr_adl_0", 
                       "encr_jcr_1", "encr_jcr_2", "encr_jcr_3", "howspoke_0", "telemode_1","telemode_2", 
                       "placeipc_1", "placeipc_2", "placeipc_3", "placeipc_4", "placeipc_5", "placeipc_6", 
                       "talkstay_0", "way_stay_0",
                       "talkvstf_0", "talktold_0", "tradwant_0", "chncetrd_1", "chncetrd_2", "chncetrd_3", 
                       "totalhrs", "vstf_cat_1", "vstf_cat_2", "schl52_0", "trng1_0", "trng2_0", 
                       "trng52_0", "welf1_0", "welf3_0", "welf4_0", "welf5_0", "welf6_0", "welf7_0", 
                       "welf8_0", "welf9_0", "welf10_0", "welf11_0", "welf12_0", "currjob_0", "nchld"]

confounders = list(set(confounders)) 

In [4]:
df = df[df['female'] == 1]  # Only women
df = df[df['treat'].isin([11,12,13,22,23,21])]
df['d'] = df['treat'].apply(lambda x: 0 if x in [11, 12, 13] else 1)

In [5]:
df_treat = df[df['d'] == 1]
df_control = df[df['d'] == 0]
s = df[['workh52_0']]
y = df[['hourwag_52']]
X = df[confounders]
d = df[['d']]

df_dml = pd.concat([s, y, X, d], axis=1)

In [6]:
dml_data = dml.DoubleMLData(data=df, x_cols=confounders, y_col='hourwag_52', d_cols='d', s_col='workh52_0')

In [ ]:
def get_ml_models(ml_model, X, y):

    if ml_model == 'lasso':

        alpha_max = np.max(np.abs(X.T @ y)) / len(y)
        #print(f"alpha_max: {alpha_max}")

        # Log-scale'de bir grid üret
        alphas = np.logspace(np.log10(alpha_max) - 4, np.log10(alpha_max), 100)

        grid_g = {'alpha': alphas}
        grid_m = {'C': np.logspace(-4, 4, 10)}
        grid_pi = {'C': np.logspace(-4, 4, 10)}

        ml_g = Lasso(max_iter=10000)
        ml_m = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000)
        ml_pi = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000)

    elif ml_model == 'rf':
        grid = {
            'max_depth': [5, 10],
            'n_estimators': [100, 300],
            'min_samples_leaf': [1, 5]
        }
        grid_g, grid_m, grid_pi = grid, grid, grid

        ml_g = RandomForestRegressor()
        ml_m = RandomForestClassifier()
        ml_pi = RandomForestClassifier()

    elif ml_model == 'xgb':

        grid_XGB = {
            'max_depth': [4, 5, 6],
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'subsample': [0.8, 1.0],
            'n_jobs': [-1]
        }

        grid_g, grid_m, grid_pi = grid_XGB, grid_XGB, grid_XGB

        ml_g = XGBRegressor()
        ml_m = XGBClassifier(eval_metric='logloss')
        ml_pi = XGBClassifier(eval_metric='logloss')

    elif ml_model == 'regression':
        
        ml_g = LinearRegression()
        ml_m = LogisticRegression()
        ml_pi = LogisticRegression()
        grid_g = None
        grid_m = None
        grid_pi = None

    return ml_g, ml_m, ml_pi, grid_g, grid_m, grid_pi


In [46]:
summaries = defaultdict(dict)
N_FOLDS = 5
TUNE_ON_FOLDS = True
SEED = 1

for tuning in ['no_tune','tune']:
    for ml_model in ['lasso', 'rf', 'xgb']:
        print(f"Running model: {ml_model}")
        np.random.seed(SEED) 
        ml_g, ml_m, ml_pi, grid_g, grid_m, grid_pi = get_ml_models(ml_model=ml_model,X=pd.concat([X,d],axis=1), y=y)
        dml_ssm = dml.DoubleMLSSM(obj_dml_data=dml_data, ml_g_d0=ml_g, ml_g_d1=ml_g, ml_m=ml_m, ml_pi=ml_pi, score='missing-at-random', n_folds=N_FOLDS)
        if tuning == 'tune':
            tune_res = dml_ssm.tune({"ml_g_d0": grid_g, "ml_g_d1": grid_g, "ml_m": grid_m, "ml_pi": grid_pi}, tune_on_folds=TUNE_ON_FOLDS, return_tune_res=True)
        dml_ssm.fit(store_predictions=True)
        summaries[tuning][ml_model] = dml_ssm.summary.to_dict()
        print(dml_ssm.summary['coef'])


Running model: lasso
d   -0.868303
Name: coef, dtype: float64
Running model: rf
d    0.633102
Name: coef, dtype: float64
Running model: xgb
d   -45.661561
Name: coef, dtype: float64
Running model: lasso


/Users/batuhan_mac/mambaforge/envs/r_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.489e-02, tolerance: 2.643e-02
  model = cd_fast.enet_coordinate_descent(
/Users/batuhan_mac/mambaforge/envs/r_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.107e-02, tolerance: 2.102e-02
  model = cd_fast.enet_coordinate_descent(
/Users/batuhan_mac/mambaforge/envs/r_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

d    0.247913
Name: coef, dtype: float64
Running model: rf
d    0.262246
Name: coef, dtype: float64
Running model: xgb
d    0.22003
Name: coef, dtype: float64


In [47]:
for tuning in ['no_tune','tune']:
    for ml_model in ['lasso', 'rf', 'xgb']:
        coef = summaries[tuning][ml_model]['coef']
        print(f"Model: {ml_model} with tuning: {tuning}, coefficient: {coef}")

Model: lasso with tuning: no_tune, coefficient: {'d': -0.8683033561700407}
Model: rf with tuning: no_tune, coefficient: {'d': 0.63310182145097}
Model: xgb with tuning: no_tune, coefficient: {'d': -45.66156051617435}
Model: lasso with tuning: tune, coefficient: {'d': 0.24791259838295357}
Model: rf with tuning: tune, coefficient: {'d': 0.2622457738203469}
Model: xgb with tuning: tune, coefficient: {'d': 0.22003043819944795}


In [48]:
flat_rows = []
TUNE_LABEL= 'of' if TUNE_ON_FOLDS else 'fs'

for tuning_type, models in summaries.items():
    for model_name, stats in models.items():
        row = {'tuning': tuning_type, 'model': model_name}
        for stat_name, stat_vals in stats.items():
            for key, val in stat_vals.items():
                row[f'{stat_name}_{key}'] = val
        flat_rows.append(row)

df_summary = pd.DataFrame(flat_rows)
df_summary.to_csv(f'dml_summary_{N_FOLDS}folds_{TUNE_LABEL}_seed{SEED}.csv', index=False)